In [1]:
import os
import sys
from importnb import Notebook
import re
import pandas as pd
import numpy as np
import zipfile
from enum import Enum
import zlib
from dataclasses import dataclass, make_dataclass, field
from typing import Optional, Union, List, Callable
from bs4 import BeautifulSoup

PKG_ROOT = os.path.dirname(os.path.realpath(os.getcwd()))
if not PKG_ROOT in sys.path:
    sys.path.append(PKG_ROOT)
    
with Notebook():
    from src.parserbase import GamCreBase, make_names, clean_duplicates, ValueBlock, SegBlock, TableBlock, BlockBase

In [2]:
class Sav(GamCreBase):
    
    def __init__(self, savefile:str, game:str='BGEE'):
        '''
        Operation at SAV level is simple, so integrated with GAM which is an implementation of GamCreBase
        Be careful of namespace of Sav class and GamCreBase class overlapping
        '''
        self.savefile = os.path.join(PKG_ROOT, 'saves', 'original', savefile)
        resourceDir = os.path.join(PKG_ROOT, 'resources', game)
        self.zipfile = zipfile.ZipFile(self.savefile)
        self.filelist = self.zipfile.namelist()
        self.files = [self.zipfile.read(_) for _ in self.filelist]
        self.gamStr = self.files[1].decode('latin')
        self.gamBuffer = bytearray(self.files[1])
        self.gamVersion = re.findall(r'GAME\s*V\d+\.\d+', self.gamStr)[0]  #always 'GAMEV2.0'
        self.creVersion = re.findall(r'CRE\s*V\d+\.\d+', self.gamStr)[0]  #always 'CRE V1.0'
        super().__init__(resourceDir=resourceDir
                         ,buffer=self.gamBuffer
                         ,valueRecords=pd.read_csv(os.path.join(resourceDir, 'GAMVALUES.csv'), index_col=0)
                         ,segRecords=pd.read_csv(os.path.join(resourceDir, 'GAMSEGS.csv'), index_col=0))  # to initiate GAM file
        
        
        

In [3]:
s = Sav('TESTSAVE.bg1save', 'BGEE')

In [4]:
gamseg_df = pd.read_csv(os.path.join(PKG_ROOT,'resources', 'bgee', 'gamsegs.csv'), index_col=0)
creseg_df = pd.read_csv(os.path.join(PKG_ROOT,'resources', 'bgee', 'cresegs.csv'), index_col=0)
cretable_df = pd.read_csv(os.path.join(PKG_ROOT,'resources', 'bgee', 'cretables.csv'), index_col=0)
res_df = pd.read_csv(os.path.join(PKG_ROOT,'resources', 'formats', 'refkeywords.csv'), index_col=0)

In [5]:
c = s.SEGS.Party.CRES[0]

In [6]:
c.TABLES.Class.refTable

,ClassCode,Class
0,1,MAGE
1,2,FIGHTER
2,3,CLERIC
3,4,THIEF
4,5,BARD
...,...,...
136,220,GREEN_DRAGON
137,221,SOD_TMP
138,222,SPECTRAL_TROLL
139,223,WIGHT
